In [1]:
import pymc3 as pm
import numpy as np
import scipy as sp
import seaborn as sns
import pandas as pd
import arviz as az
from matplotlib import pyplot as plt

In [2]:
import nbsetup
from models.sparc.dataframe import load_analysis

In [3]:
all_data = load_analysis()

AttributeError: Can't get attribute 'SparcMassProfile' on <module 'models.sparc' from '/Users/timjdavey/Projects/darkness/relativity/notebooks/../models/sparc/__init__.py'>

In [ ]:
coords = {
    "galaxy": df.Galaxy.unique(),
    "observation": df.Vobs.index
}

gamma_mu = 6
alpha_mu = 0.15

with pm.Model(coords=coords) as galaxy_model:
    
    # Universal priors
    gamma = pm.Normal('gamma', gamma_mu)
    alpha = pm.Normal('alpha', alpha_mu)
    
    # Galaxy priors
    DegreesNormal = pm.Bound(pm.Normal, lower=0.0, upper=90.0)
    inc = DegreesNormal('inclination', mu=FROMSPARC, sigma=FROMSPARC, dims='galaxy')
    PositiveNormal = pm.Bound(pm.Normal, lower=0.0)
    dist = pm.PositiveNormal('distance', mu=FROMSPARC, sigma=FROMSPARC, dims='galaxy')
    #tau = pm.Exponential('tau field', 1)
    sigma = pm.HalfCauchy("sigma", beta=FROMSPARC, dims="galaxy") # observational error

    # Data
    f = pm.Data("f", df.x_vec, dims="observation")
    r = pm.Data("r", df.R, dims="observation")
    nu = pm.Data("nu", df.nulled, dims="observation")
    g = pm.Data("g", df.gidx, dims="observation")
    
    # Prediction model
    ftd = f*(gamma/(1+(nu)**alpha)) # adjust for nulled field
    vtd = np.sqrt(ftd*r*dist[g]/D_FROMSPARC) # adjust r for distance when calc V
    vinc = vtd*np.sin(inc[g]*np.pi/180)/np.sin(INC_FROMSPARC*np.pi/180) # adjust for inclination of Vobs
    Vpred = pm.Deterministic("prediction", vinc, dims="observation")
    
    # Define likelihood
    obs = pm.Normal("obs", mu=Vpred, sigma=sigma[g], observed=df.Vobs, dims="observation")